In [34]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import ta
import matplotlib.pyplot as plt
import vnstock as vn
from datetime import timedelta
from vnstock3 import Vnstock

# Setting

In [35]:
rf = {'2019': 0.0451, '2020': 0.0286, '2021':0.023,'2022': 0.0335,'2023':0.0321}
rf = pd.Series(rf)
rf_cal = rf.mean()
rf_2022 = 0.0335

In [36]:
pd.set_option('display.max_columns', None) 

In [37]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
OBV_PERIOD = 5
initial_investment = 100000000


In [38]:
win_rate = 0.7070060208
loss_rate = 1 - win_rate
mean_profit = 0.1721618831
mean_loss = 0.1082770633
stop_loss = 0.08

In [39]:
def kelly_criterion(p, q, profit, loss):
    b = (profit * 160_000_000)/(loss * 160_000_000)
    f = (b*p - q)/b
    return f

In [40]:
f =kelly_criterion(win_rate,loss_rate,mean_profit,mean_loss)

In [41]:
high_ESG_group = ['CTD', 'DHG', 'DPM', 'FPT', 'GAS', 'MBB']
low_ESG_group = ['NVL', 'PNJ', 'REE', 'SBT', 'SSI', 'STB', 'VIC', 'VNM']
non_ESG_group = ['CII', 'CTG', 'EIB', 'GMD', 'HDB', 'HPG', 'MSN', 'MWG', 'ROS', 'SAB', 'TCB', 'VCB', 'VHM', 'VJC', 'VPB', 'VRE']


In [42]:
companies = ['VCB']

# Calculate indicators

In [43]:
def calculate_indicators(df):
    if df.empty:
        return df
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Bollinger_high'] = ta.volatility.bollinger_hband(df['close'], window=15, window_dev=2)
    df['Bollinger_low'] = ta.volatility.bollinger_lband(df['close'], window=15, window_dev=2)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

In [44]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['close'] <= df['Bollinger_low']) &
        (df['RSI'] <= RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] >= RSI_OVERBOUGHT) &
        (df['close'] >= df['Bollinger_high']), 'Signal'] = -1

    return df

In [45]:
def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

In [46]:
def calculate_daily_risk_free_rate(annual_rate_decimal, trading_days=252):
    # Convert percentage to decimal
    # Calculate daily risk-free rate using the compound formula
    daily_rate = (1 + annual_rate_decimal) ** (1 / trading_days) - 1
    
    # Alternatively, for a simple calculation:
    # daily_rate = annual_rate_decimal / trading_days
    
    return daily_rate

# Example usage
annual_yield = 0.0326  # Example annual yield of 3%
daily_risk_free_rate = calculate_daily_risk_free_rate(rf_cal)
daily_risk_free_rate_2022 = calculate_daily_risk_free_rate(rf_2022)


# Backtest

In [47]:
def simulate_investment(
    ticker, win_rate, loss_rate, mean_profit, mean_loss, 
    sell_fraction, start_date, end_date, f_star=1
):
    try:
        # Initialize trade counters and portfolio metrics
        number_of_buying_trades = 0
        number_of_selling_trades = 0
        cash = initial_investment
        holdings = 0
        portfolio_values = []

        # Load stock data and calculate indicators
        data = vn.stock_historical_data(ticker, start_date, end_date, resolution='1D', type='stock', source='TCBS')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = macd_strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i, current_date in enumerate(data.index):
            current_price = data['close'].iloc[i]

            # Handle pending T+2 settlements
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            # Avoid trades in January 2024
            if current_date.month == 1 and current_date.year == 2024:
                portfolio_values.append(cash + holdings * current_price)
                continue

            # Buy if there's a buy signal and cash allows
            if current_date in buy_signals:
                allocation = cash * f_star
                shares_to_buy = int(allocation // current_price)
                total_cost = shares_to_buy * current_price
                if shares_to_buy > 0 and cash >= total_cost:
                    cash -= total_cost
                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    pending_buy_shares[settlement_date] = pending_buy_shares.get(settlement_date, 0) + shares_to_buy
                    last_buy_price = current_price
                    number_of_buying_trades += 1

            # Sell if there's a sell signal and holdings allow
            if holdings > 0 and current_date in sell_signals:
                shares_to_sell = int(holdings * sell_fraction)
                revenue = shares_to_sell * current_price
                holdings -= shares_to_sell
                settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                pending_sell_revenue[settlement_date] = pending_sell_revenue.get(settlement_date, 0) + revenue
                number_of_selling_trades += 1

            # Update portfolio value
            portfolio_values.append(cash + holdings * current_price)

        # Finalize portfolio values including pending settlements
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)
            portfolio_values.append(cash + holdings * data['close'].iloc[-1])
            final_date += BDay(1)

        # Adjust portfolio values to match data index length
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        # Add portfolio values to data frame
        data['Portfolio_Value'] = portfolio_values
        data['value'] = portfolio_values
        data['Number_of_Buying_Trades'] = number_of_buying_trades
        data['Number_of_Selling_Trades'] = number_of_selling_trades
        # Ensure the column 'value' is directly modified in the original DataFrame
        data['value'] = data['value'].apply(lambda x: np.nan if pd.notna(x) and x < 1000000 else x)
        data['value'] = data['value'].ffill()

        data['Daily_Return'] = data['value'].pct_change()
        data['Accumulated_Profit'] = data['value'] - initial_investment

        data['Running_Max'] = data['value'].cummax()  # Track the running max portfolio value
        data['Drawdown'] = (data['value'] - data['Running_Max']) / data['Running_Max']  # Calculate drawdown
        data = data.dropna(subset=['time'])
        return data
        

    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()


In [48]:
def calculate_sharpe_ratio(data, risk_free_rate=0.01):
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Daily_Return'].dropna()

    # Calculate average return and standard deviation of returns
    average_return = daily_returns.mean()
    std_deviation = daily_returns.std()

    # Calculate the Sharpe Ratio
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation if std_deviation > 0 else np.nan

    return sharpe_ratio

In [49]:
def calculate_sortino_ratio(data,rf=0.01):  # Target return can be set to risk-free rate
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Daily_Return'].dropna()

    # Calculate average return
    average_return = daily_returns.mean()

    # Calculate downside returns (returns below the target return)
    downside_returns = daily_returns[daily_returns < rf]

    # Calculate downside deviation
    downside_deviation = downside_returns.std() if not downside_returns.empty else np.nan

    # Calculate the Sortino Ratio
    sortino_ratio = (average_return - rf) / downside_deviation if downside_deviation > 0 else np.nan

    return sortino_ratio

thêm phần điều chỉnh f star ở code phía dưới

In [50]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, \
                                sell_fraction, start_date, end_date, rf):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, mean_loss, \
                                     sell_fraction, start_date=start_date, end_date=end_date, f_star=1)
        if not result.empty:
            # Calculate the Sharpe Ratio for the result
            sharpe_ratio = calculate_sharpe_ratio(result,risk_free_rate=rf)
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / initial_investment * 100,
                'Number of Buying Trades': result['Number_of_Buying_Trades'].max(),
                'Number of Selling Trades': result['Number_of_Selling_Trades'].max(),
                'Sharpe Ratio': sharpe_ratio,
                'Sortino Ratio': calculate_sortino_ratio(result,rf=rf),
                'MDD': result['Drawdown'].min(),
            })
    return pd.DataFrame(results)

# Kelly 2019-2024

In [51]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss,\
                                        sell_fraction, start_date='2018-12-28' ,end_date='2024-01-05', rf=daily_risk_free_rate)
high_esg = pd.DataFrame(results_df)
print(high_esg)
average_rate_of_return = results_df['Rate of Return'].mean()
average_profit = results_df[results_df['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = results_df[results_df['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}','avg sharpe:',results_df['Sharpe Ratio'].mean(),'avg sortino:',results_df['Sortino Ratio'].mean())

Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD               75073640   -24926360.0      -24.926360   
1     DHG              128400990    28400990.0       28.400990   
2     DPM              305468010   205468010.0      205.468010   
3     FPT              198609830    98609830.0       98.609830   
4     GAS              132009969    32009969.0       32.009969   
5     MBB               89043120   -10956880.0      -10.956880   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        7                         6     -0.005008   
1                        5                         4      0.012170   
2              

In [52]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss,\
                                             sell_fraction, start_date='2018-12-29', end_date='2024-01-05', \
                                                rf=daily_risk_free_rate)
low_esg_kelly = pd.DataFrame(low_esg_kelly)
print(low_esg_kelly)
average_rate_of_return = low_esg_kelly['Rate of Return'].mean()
average_profit = low_esg_kelly[low_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly[low_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL               32445770   -67554230.0      -67.554230   
1     PNJ              123863720    23863720.0       23.863720   
2     REE              175281940    75281940.0       75.281940   
3     SBT              108039280     8039280.0        8.039280   
4     SSI               71800410   -28199590.0      -28.199590   
5     STB              152577950    52577950.0       52.577950   
6     VIC              103199686     3199686.0        3.199686   
7     VNM              123440950    23440950.0       23.

In [53]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2018-12-29', end_date='2024-01-05', rf=daily_risk_free_rate)
non_esg_kelly = pd.DataFrame(non_esg_kelly)
print(non_esg_kelly)
average_rate_of_return = non_esg_kelly['Rate of Return'].mean()
average_profit = non_esg_kelly[non_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly[non_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
   Company  Final Portfolio Value  Total Profit  Rate of Return  \
0      CII               82751910   -17248090.0      -17.248090   
1      CTG              262339000 

In [54]:
high_esg['ESG'] ='High'
low_esg_kelly['ESG'] = 'Low'
non_esg_kelly['ESG'] = 'Non'

In [55]:
merged = pd.concat([high_esg, low_esg_kelly, non_esg_kelly]) 
merged['period'] = '2019-2024' 

# Kelly 2022

In [56]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
high_esg_2022 = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=daily_risk_free_rate_2022)
high_esg_2022 = pd.DataFrame(high_esg_2022)
print(high_esg_2022)
average_rate_of_return = high_esg_2022['Rate of Return'].mean()
average_profit = high_esg_2022[high_esg_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = high_esg_2022[high_esg_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
high_esg_2022['ESG'] ='High'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD               52298470   -47701530.0       -47.70153   
1     DHG               88447320   -11552680.0       -11.55268   
2     DPM              161199240    61199240.0        61.19924   
3     FPT              117468600    17468600.0        17.46860   
4     GAS              100000000           0.0         0.00000   
5     MBB               78882750   -21117250.0       -21.11725   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        4                         1     -0.070460   
1                        2                         0     -0.049810   
2                    

In [57]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly_2022 = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=daily_risk_free_rate_2022)
low_esg_kelly_2022 = pd.DataFrame(low_esg_kelly_2022)
print(low_esg_kelly_2022)
average_rate_of_return = low_esg_kelly_2022['Rate of Return'].mean()
average_profit = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
low_esg_kelly_2022['ESG'] = 'Low'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL               19617200   -80382800.0      -80.382800   
1     PNJ              106241500     6241500.0        6.241500   
2     REE              100000000           0.0        0.000000   
3     SBT               72229960   -27770040.0      -27.770040   
4     SSI               65125820   -34874180.0      -34.874180   
5     STB               76898400   -23101600.0      -23.101600   
6     VIC               70845304   -29154696.0      -29.154696   
7     VNM               99274730     -725270.0       -0.725270  

In [58]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly_2022 = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=daily_risk_free_rate_2022)
non_esg_kelly_2022 = pd.DataFrame(non_esg_kelly_2022)
print(non_esg_kelly_2022)
average_rate_of_return = non_esg_kelly_2022['Rate of Return'].mean()
average_profit = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
non_esg_kelly_2022['ESG'] = 'Non'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
   Company  Final Portfolio Value  Total Profit  Rate of Return  \
0      CII               69458100   -30541900.0       -30.54190   
1      CTG               85163250   -14836750.0   

In [59]:
merged_2022 = pd.concat([high_esg_2022, low_esg_kelly_2022, non_esg_kelly_2022])
merged_2022['period'] = '2022-2023'
merged_2022

,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,CTD,52298470,-47701530.0,-47.701530,4,1,-0.070460,-0.089315,-0.599494,High,2022-2023
1,DHG,88447320,-11552680.0,-11.552680,2,0,-0.049810,-0.072122,-0.209542,High,2022-2023
2,DPM,161199240,61199240.0,61.199240,2,1,0.089001,0.133622,-0.359973,High,2022-2023
3,FPT,117468600,17468600.0,17.468600,3,1,0.045652,0.060489,-0.181798,High,2022-2023
4,GAS,100000000,0.0,0.000000,0,0,NaN,NaN,0.000000,High,2022-2023
5,MBB,78882750,-21117250.0,-21.117250,3,1,-0.033392,-0.040544,-0.362777,High,2022-2023
0,NVL,19617200,-80382800.0,-80.382800,3,1,-0.264857,-0.285886,-0.822712,Low,2022-2023
1,PNJ,106241500,6241500.0,6.241500,1,1,0.016844,0.020188,-0.118641,Low,2022-2023
2,REE,100000000,0.0,0.000000,0,0,NaN,NaN,0.000000,Low,2022-2023
3,SBT,72229960,-27770040.0,-27.770040,1,1,-0.046362,-0.059836,-0.482833,Low,2022-2023


# Gộp 2 cái

In [60]:
merged_all = pd.concat([merged, merged_2022])
merged_all= merged_all.reset_index()
merged_all['total_trades']= merged_all['Number of Buying Trades'] + merged_all['Number of Selling Trades']
merged_all.to_csv('macd kelly.csv', index=False)

In [61]:
merged_all['total_trades']= merged_all['Number of Buying Trades'] + merged_all['Number of Selling Trades']
merged_all.to_csv('boll no kelly.csv', index=False)

In [62]:
A= merged_all.groupby(['ESG','period'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].mean()
A

Rate of Return  Sharpe Ratio  Sortino Ratio  total_trades  \
ESG  period                                                                 
High 2019-2024       54.767593      0.019352       0.029605     10.333333   
     2022-2023       -0.283937     -0.003802      -0.001574      3.000000   
Low  2019-2024       11.331213      0.002342       0.005577     10.375000   
     2022-2023      -23.720886     -0.065419      -0.077500      2.250000   
Non  2019-2024       23.211624      0.003727       0.007179     10.125000   
     2022-2023       -8.784751     -0.008387       0.027665      2.375000   

                     MDD  
ESG  period               
High 2019-2024 -0.406857  
     2022-2023 -0.285597  
Low  2019-2024 -0.486530  
     2022-2023 -0.383760  
Non  2019-2024 -0.466784  
     2022-2023 -0.286757

In [63]:
merged_all.groupby(['ESG','period'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].median()


Rate of Return  Sharpe Ratio  Sortino Ratio  total_trades  \
ESG  period                                                                 
High 2019-2024       30.205479      0.012944       0.019394          10.5   
     2022-2023       -5.776340     -0.033392      -0.040544           3.5   
Low  2019-2024       15.740115      0.007869       0.011083          10.0   
     2022-2023      -25.435820     -0.043168      -0.059050           2.0   
Non  2019-2024        6.287920      0.002889       0.004050          10.0   
     2022-2023      -10.800540     -0.022216      -0.029983           2.0   

                     MDD  
ESG  period               
High 2019-2024 -0.379196  
     2022-2023 -0.284757  
Low  2019-2024 -0.464310  
     2022-2023 -0.450753  
Non  2019-2024 -0.455647  
     2022-2023 -0.282770

In [64]:
kelly = pd.read_csv('boll kelly.csv')
no_kelly = pd.read_csv('boll no kelly.csv')

In [65]:
kelly['Kelly'] = 'Yes'
no_kelly['Kelly'] = 'No'
double_merged = pd.concat([kelly, no_kelly])
double_merged.to_excel('boll.xlsx', index=False)

In [66]:
B= double_merged.groupby(['ESG','period','Kelly'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].mean()
B.to_excel('boll_double_merged.xlsx')